# How to run

Please use the requirements.txt and after creating your virtual env :

`venv venv_env_name`

and install the necessary env 

`pip install -r requirements.txt `

create a folder data and copy the excercise.db to it. 

I create the connection to our database 

In [1]:
import sqlite3
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


conn = sqlite3.connect('data/exercise.db')
cursor = conn.cursor()


# Tasks 
Read in the data from database.
 

In [2]:
query = """SELECT *  FROM events"""
df = pd.read_sql_query (query , conn)

#### Task1
1. What’s the total number of users present in the dataset? Total number of unique users.

In [3]:
df.user_id.nunique()

1586

#### Task2 
2. List the number of installs per country.

In [4]:
df.groupby('country')['user_id'].nunique()

country
Austria         193
Germany        1273
Switzerland     124
Name: user_id, dtype: int64

for this task I need to first bring the timestamp to germany timezone and make it into datetime format. 

In [5]:
df['german_datetime']= pd.to_datetime(df['event_timestamp'], unit='us' , utc= True  ).dt.tz_convert('Europe/Berlin')
df['german_datetime'].head()

0   2022-08-08 01:04:25.766181+02:00
1   2022-08-08 01:04:26.183184+02:00
2   2022-08-08 01:06:28.094199+02:00
3   2022-08-08 01:06:28.581202+02:00
4   2022-08-08 01:08:13.624213+02:00
Name: german_datetime, dtype: datetime64[ns, Europe/Berlin]

#### Installation_date
I will then calculate the installation_date , I will assume the installation date is the same as the first activity. 

In [6]:
df['installation_date'] = df.groupby('user_id')['german_datetime'].transform('first').dt.date


Creating the conditions :

In [7]:

condition_os = (df['os'] =='Android')
condition_date = (df['installation_date'].astype(str) == '2022-08-02') 
condition_country = (df['country'] =='Germany')

#### TASK 3.1
 How many users installed the app on August 2, 2022 in Germany on Android?

In [8]:
df[condition_os & condition_date & condition_country].user_id.nunique()


92

#### TASK 3.2 and 3.3

How many of these users are active on the first, third, and fourteenth day after the install respectively? (I.e., count users for all three days separately)

In [9]:
cohort_users= df[condition_os & condition_date & condition_country].user_id.to_list()


In [10]:
df['acitivy_days_after_installation'] = ((df['german_datetime'].dt.date - df['installation_date'])/ np.timedelta64(1, 'D')).astype(int)
df.head()

,platform,country,event_timestamp,event_name,user_id,os,ad_revenue,tracker_name,german_datetime,installation_date,acitivy_days_after_installation
0,Native,Germany,1.659913e+15,played_game,99f907fd3e37cbc92c9af6663a3dfd0b2ca6cad9,Android,0.000000,Unattributed,2022-08-08 01:04:25.766181+02:00,2022-08-08,0
1,Native,Germany,1.659913e+15,watched_ad,99f907fd3e37cbc92c9af6663a3dfd0b2ca6cad9,Android,0.003952,Unattributed,2022-08-08 01:04:26.183184+02:00,2022-08-08,0
2,Native,Germany,1.659914e+15,played_game,99f907fd3e37cbc92c9af6663a3dfd0b2ca6cad9,Android,0.000000,Unattributed,2022-08-08 01:06:28.094199+02:00,2022-08-08,0
3,Native,Germany,1.659914e+15,watched_ad,99f907fd3e37cbc92c9af6663a3dfd0b2ca6cad9,Android,0.018284,Unattributed,2022-08-08 01:06:28.581202+02:00,2022-08-08,0
4,Native,Germany,1.659914e+15,played_game,99f907fd3e37cbc92c9af6663a3dfd0b2ca6cad9,Android,0.000000,Unattributed,2022-08-08 01:08:13.624213+02:00,2022-08-08,0


In [11]:
df_cohort = df[df.user_id.isin(cohort_users)]


In [12]:
counts = df_cohort['acitivy_days_after_installation'].value_counts().sort_index() 
percentages = df_cohort['acitivy_days_after_installation'].value_counts(normalize=True).sort_index()* 100
result_df = pd.concat([counts, percentages], axis=1, keys=['Count', 'Percentage'] ).reset_index()
result_df

,acitivy_days_after_installation,Count,Percentage
0,0,1454,55.327245
1,1,728,27.701674
2,2,255,9.703196
3,3,151,5.745814
4,4,34,1.293760
5,5,4,0.152207
6,6,2,0.076104


since we don't need all of these values only 1 day and 3 day retentions.

In [13]:
result_df[result_df.acitivy_days_after_installation.isin([1, 3 , 14 ])]

,acitivy_days_after_installation,Count,Percentage
1,1,728,27.701674
3,3,151,5.745814


As seen above we don't have any users that come back after 14 days. Just to check one more time if that is true, I look at the max date that I have. 

In [14]:
max(df.german_datetime)

Timestamp('2022-08-09 01:59:57.039074500+0200', tz='Europe/Berlin')

ince the data in the database only goes until 2022-08-09, the available data is not sufficient to calculate a 14-day return window.

#### Task 4 . 
I have created a query.sql for creating this view. I usually prefer ctes over nested windows function as they make the code more readable and manageable. 
#### installation 
In the installation CTE I created the installation date for our unique users. 

#### enriched_events
basically I put the installation CTE together with my events, and created the human readable format. 

#### daily_stats
Created the number of installs for a given installation date  and sum for ad_revenue.

#### Marketing View
Join our daily_stats with user_acquisition and creating our marketing view



In [15]:
with open('marketing_view.sql', 'r') as sql_file:
    sql_query = sql_file.read()

Dropping any old views that might have been created prior.

In [16]:
cursor.execute("DROP VIEW IF EXISTS marketing")

In [17]:
cursor.execute(sql_query)


Checking if the view was created : 


In [18]:
df_marketing = pd.read_sql_query("SELECT * FROM marketing" , conn)
df_marketing.head()

,date,tracker_name,costs,number_of_installs,total_revenue
0,2022-08-02,google_campaign1,73.058302,26,11.080362
1,2022-08-02,google_campaign2,3.872444,2,0.482148
2,2022-08-02,meta_campaign1,14.294755,6,1.533749
3,2022-08-02,unity_campaign1,5.923850,2,1.860135
4,2022-08-03,google_campaign1,63.722366,36,11.067812


Double checking if the numbers match 

In [19]:
df_marketing[(df_marketing["date"].astype(str) == '2022-08-06' ) & (df_marketing["tracker_name"] =='google_campaign1')].number_of_installs


16    33
Name: number_of_installs, dtype: int64

In [20]:
df[(df.installation_date.astype(str) == '2022-08-06' ) & (df.tracker_name =='google_campaign1')].user_id.nunique()


33

# Task5 
 Query the view marketing and report the Costs per Install (CPI) on August
6, 2022, for campaign “google_campaign1”?

In [21]:
task5_query="""
SELECT 
costs / number_of_installs AS costs_per_install 
FROM 
    marketing
WHERE  
    date = '2022-08-06' 
    AND tracker_name = 'google_campaign1'
"""

In [22]:
cost_per_install = pd.read_sql_query(task5_query , conn)
cost_per_install

,costs_per_install
0,3.430289


Closing connections

In [ ]:
conn.close()


### Remarks :
This code was made for a short ad-hoc analysis task only.
It is not optimized or structured as production code.

### Future improvment :
- Create a .py file with functions to read, query and filter data. 
- Add error handling. 
- Create a shareable output (for example Excel) so the results can be reused by others.
- Create a dashboard 